"""
趋势增强型智能网格策略 v4.0 (全面修复优化版)
修复内容：
1. 网格中心更新逻辑缺陷 - 价格对齐与盈利安全检查
2. 趋势加仓逻辑 - 多因子确认机制
3. 仓位动态平衡 - 金字塔式网格仓位管理
4. 止损后渐进式重启 - 分批建仓机制
5. 可视化索引对齐 - 基于merge_asof的时间对齐
6. 风险管理 - 流动性储备与跨时间框架验证
7. 绩效指标 - Sortino Ratio、盈亏比等高级指标
8. 波动率自适应网格 - 动态间距调整
9. 跨时间框架验证 - 周线趋势过滤
10. 动态止损 - ATR-based Chandelier Exit
"""

#主要修复与优化总结
#核心修复
#网格中心更新：增加价格比例调整与盈利安全检查，避免转移后无法盈利
#时间对齐：使用 pd.merge_asof 替代索引对齐，确保时间序列安全匹配
#卖出逻辑：保持先保存数据再清零的修复
#关键增强
#金字塔仓位：层级越高（价格越低）仓位越大，但用平方根控制增速
#多因子加仓：增加周线趋势、成交量、RSI、波动率四重过滤
#渐进式重启：止损后分3批建仓，每批间隔5天，降低一次性重仓风险
#双止损机制：固定比例止损 + ATR-based Chandelier Exit
#自适应网格：每20天检查波动率变化，动态调整间距
#流动性管理：强制保留10%现金储备，防止满仓套牢
#新增指标
#Sortino Ratio：只惩罚下行波动
#Profit Factor：总盈利/总亏损
#Win/Loss Ratio：平均盈利/平均亏损
#连续盈亏统计：最大连续盈利/亏损次数
#平均持仓天数
#参数调整建议
#网格层数：7→5（降低深度套牢风险）
#基础间距：2.5%→3.5%（减少震荡市交易频率）
#止损线：15%→12%（更严格风控）
#冷却期：20→12天（更快恢复交易）


In [2]:
import pandas as pd
import numpy as np
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from pathlib import Path
import warnings
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from numba import jit

ModuleNotFoundError: No module named 'numba'

In [4]:
warnings.filterwarnings('ignore')

In [5]:
# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

In [6]:
@dataclass(slots=True)
class Trade:
    """交易记录"""
    date: pd.Timestamp
    type: str
    price: float
    shares: float
    value: float
    pnl: Optional[float] = None
    grid_level: Optional[int] = None

In [7]:
@dataclass
class StrategyConfig:
    """策略配置参数"""
    initial_capital: float = 100000.0
    base_grid_pct: float = 0.035  # 调整为3.5%
    grid_levels: int = 5  # 调整为5层
    max_position: float = 0.9
    stop_loss: float = 0.12  # 调整为12%
    trend_period: int = 20
    vol_period: int = 14
    cooldown_days: int = 12  # 调整为12天
    min_grid_spacing: float = 0.015
    max_grid_spacing: float = 0.08
    position_update_threshold: float = 0.08  # 调整为8%
    min_cash_reserve: float = 0.1  # 最小现金储备10%
    weekly_trend_period: int = 20  # 周线趋势周期
    atr_stop_multiplier: float = 2.5  # ATR止损倍数
    rebalance_interval: int = 20  # 网格重平衡间隔


In [8]:
class GridManager:
    """网格管理器 - 修复中心更新逻辑"""
    
    def __init__(self, center_price: float, levels: int, spacing: float):
        self.center_price = center_price
        self.levels = levels
        self.spacing = spacing
        self.grids: List[Dict] = []
        self._init_grids()
    
    def _init_grids(self):
        """初始化网格"""
        self.grids = []
        for i in range(1, self.levels + 1):
            self.grids.append({
                'level': i,
                'buy_price': self.center_price * (1 - self.spacing * i),
                'sell_price': self.center_price * (1 + self.spacing * i),
                'active': False,
                'shares': 0.0,
                'entry_price': 0.0,
                'entry_date': None
            })
    
    def update_center(self, new_center: float, spacing: float, 
                     preserve_active: bool = False) -> List[Dict]:
        """
        修复版：更新网格中心，确保价格对齐和盈利安全
        """
        closed_grids = []
        
        if preserve_active:
            old_grids = {g['level']: g for g in self.grids if g['active']}
            old_center = self.center_price
            
            # 计算价格偏移比例
            price_ratio = new_center / old_center if old_center > 0 else 1.0
            
            self.center_price = new_center
            self.spacing = spacing
            self._init_grids()
            
            for level, old_grid in old_grids.items():
                if level > self.levels:
                    # 层级超出范围，强制平仓
                    closed_grids.append(old_grid)
                    continue
                
                new_grid = self.grids[level - 1]
                adjusted_entry = old_grid['entry_price'] * price_ratio
                
                # 盈利安全检查：确保卖出价高于调整后的成本价（至少0.1%盈利空间）
                min_profit_price = adjusted_entry * 1.001
                
                if new_grid['sell_price'] <= min_profit_price:
                    # 网格移动导致无法盈利，强制平仓
                    closed_grids.append(old_grid)
                    continue
                
                # 安全转移持仓
                new_grid['active'] = True
                new_grid['shares'] = old_grid['shares']
                new_grid['entry_price'] = adjusted_entry
                new_grid['entry_date'] = old_grid.get('entry_date')
        else:
            closed_grids = [g for g in self.grids if g['active']]
            self.center_price = new_center
            self.spacing = spacing
            self._init_grids()
        
        return closed_grids
    
    def get_active_grids(self) -> List[Dict]:
        """获取已激活的网格"""
        return [g for g in self.grids if g['active']]
    
    def get_inactive_buy_grids(self) -> List[Dict]:
        """获取未激活的买入网格"""
        inactive = [g for g in self.grids if not g['active']]
        return sorted(inactive, key=lambda x: x['buy_price'], reverse=True)
    
    def check_signals(self, high: float, low: float) -> Tuple[List[Dict], List[Dict]]:
        """检查交易信号"""
        buy_signals = []
        sell_signals = []
        
        for grid in self.grids:
            if not grid['active'] and low <= grid['buy_price']:
                buy_signals.append(grid)
            elif grid['active'] and high >= grid['sell_price']:
                sell_signals.append(grid)
        
        return buy_signals, sell_signals

In [9]:
class TrendEnhancedGridStrategy:
    """
    趋势增强型智能网格策略 v4.0
    全面修复优化版
    """
    
    def __init__(self, config: Optional[StrategyConfig] = None):
        self.config = config or StrategyConfig()
        self.reset()
    
    def reset(self):
        """重置策略状态"""
        self.cash = self.config.initial_capital
        self.position = 0.0
        self.cost_basis = 0.0
        self.grid_manager: Optional[GridManager] = None
        self.trades: List[Trade] = []
        self.values: List[Dict] = []
        self.max_value = self.config.initial_capital
        self.cooldown_counter = 0
        self.last_grid_update_idx = 0
        self._last_add_idx = 0
        self._entry_max_price = 0.0  # 用于ATR止损
        
        # 渐进式重启状态
        self._restart_phase = 0
        self._restart_base_cash = 0.0
        self._last_restart_idx = 0
        
        # 预计算数据
        self._df: Optional[pd.DataFrame] = None
        self._current_idx: int = 0
    
    def prepare_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """预计算所有技术指标，增加周线趋势"""
        df = df.copy()
        
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.sort_values('datetime').reset_index(drop=True)
        
        # 处理成交量
        if df['Volume'].dtype == object:
            def parse_volume(v):
                if isinstance(v, str):
                    v = v.strip()
                    if 'B' in v.upper(): 
                        return float(v.replace('B', '').replace('b', '')) * 1e9
                    elif 'M' in v.upper(): 
                        return float(v.replace('M', '').replace('m', '')) * 1e6
                    elif 'K' in v.upper():
                        return float(v.replace('K', '').replace('k', '')) * 1e3
                return float(v)
            df['Volume'] = df['Volume'].apply(parse_volume)
        
        # 计算RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # 计算ATR
        high_low = df['High'] - df['Low']
        high_close = (df['High'] - df['Close'].shift()).abs()
        low_close = (df['Low'] - df['Close'].shift()).abs()
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['ATR'] = tr.rolling(window=self.config.vol_period).mean()
        
        # 计算均线
        df['MA5'] = df['Close'].rolling(5).mean()
        df['MA10'] = df['Close'].rolling(10).mean()
        df['MA20'] = df['Close'].rolling(self.config.trend_period).mean()
        
        # 周线趋势（20日均线平滑后再4日平滑，近似周线）
        df['Weekly_MA'] = df['Close'].rolling(self.config.weekly_trend_period).mean()
        df['Weekly_MA_Slow'] = df['Weekly_MA'].rolling(4).mean()
        df['Weekly_Trend'] = np.where(df['Weekly_MA'] > df['Weekly_MA_Slow'], 1, -1)
        
        # 日线趋势评分
        conditions = [
            (df['Close'] > df['MA5']) & (df['MA5'] > df['MA10']) & (df['MA10'] > df['MA20']),
            (df['Close'] < df['MA5']) & (df['MA5'] < df['MA10']) & (df['MA10'] < df['MA20'])
        ]
        
        bull_score = ((df['Close'] / df['MA20'] - 1) * 5).clip(upper=1.0)
        bear_score = ((df['Close'] / df['MA20'] - 1) * 5).clip(lower=-1.0)
        range_score = ((df['Close'] - df['MA20']) / df['MA20'] * 3).clip(-0.3, 0.3)
        
        df['Trend'] = np.select(conditions, [bull_score, bear_score], default=range_score)
        
        # RSI调整
        rsi_overbought = df['RSI'] > 75
        rsi_oversold = df['RSI'] < 25
        df.loc[rsi_overbought, 'Trend'] -= 0.3
        df.loc[rsi_oversold, 'Trend'] += 0.3
        df['Trend'] = df['Trend'].clip(-1.0, 1.0)
        
        # 成交量
        df['Volume_MA20'] = df['Volume'].rolling(20).mean()
        
        # 波动率（用于自适应网格）
        df['Volatility'] = df['Close'].pct_change().rolling(20).std()
        
        return df
    
    def get_grid_spacing(self, atr: float, price: float) -> float:
        """基于ATR计算动态网格间距"""
        if atr <= 0 or price <= 0:
            return self.config.base_grid_pct
        
        volatility = atr / price
        spacing = volatility * 2.5
        
        return max(self.config.min_grid_spacing, 
                  min(self.config.max_grid_spacing, spacing))
    
    def get_position_size(self, total_value: float, price: float,
                         is_trend_add: bool = False, grid_level: int = 0) -> float:
        """
        金字塔式仓位管理：层级越高（价格越低），仓位越大
        """
        if total_value <= 0 or price <= 0:
            return 0.0
        
        # 基础仓位
        base_size = 0.15
        
        if not is_trend_add and grid_level > 0:
            # 网格交易：层级越高，仓位越大（平方根控制）
            # level 1: 0.15 * sqrt(1)/2 = 0.075, level 5: 0.15 * sqrt(5)/2 ≈ 0.168
            level_multiplier = np.sqrt(grid_level) / 2
            base_size *= (1 + level_multiplier)
        
        # 趋势增强
        if is_trend_add:
            current_trend = self._df['Trend'].iloc[self._current_idx]
            base_size *= (1 + max(0, current_trend) * 0.5)
        
        # 波动率调整
        atr = self._df['ATR'].iloc[self._current_idx]
        if atr > 0 and price > 0:
            vol = atr / price
            base_size /= (1 + vol * 5)
        
        # 检查仓位限制
        current_pct = self.position * price / total_value
        available = self.config.max_position - current_pct
        
        # 保留现金储备
        cash_reserve = total_value * self.config.min_cash_reserve
        available_cash = self.cash - cash_reserve
        max_by_cash = available_cash / total_value if total_value > 0 else 0
        
        return max(0.0, min(base_size, available, max_by_cash, 0.25))
    
    def execute_buy(self, grid: Optional[Dict], price: float, date: pd.Timestamp,
                   total_value: float, is_trend_add: bool = False) -> bool:
        """执行买入"""
        grid_level = grid['level'] if grid else 0
        size = self.get_position_size(total_value, price, is_trend_add, grid_level)
        
        if size <= 0:
            return False
        
        # 趋势乘数
        current_trend = self._df['Trend'].iloc[self._current_idx]
        trend_multiplier = 1 + max(0, current_trend) * 0.5
        invest = total_value * size * trend_multiplier
        
        # 现金检查（保留储备金）
        cash_reserve = total_value * self.config.min_cash_reserve
        available_cash = self.cash - cash_reserve
        
        max_shares_by_cash = available_cash / price if price > 0 else 0
        shares = min(invest / price, max_shares_by_cash) if price > 0 else 0
        
        if shares <= 0:
            return False
        
        cost = shares * price
        self.cash -= cost
        self.position += shares
        
        # 更新成本基础
        if self.position > 0:
            total_cost = self.cost_basis * (self.position - shares) + cost
            self.cost_basis = total_cost / self.position
        else:
            self.cost_basis = price
        
        # 更新入场最高价（用于ATR止损）
        self._entry_max_price = max(self._entry_max_price, price)
        
        # 更新网格状态
        if grid:
            grid['active'] = True
            grid['shares'] = shares
            grid['entry_price'] = price
            grid['entry_date'] = date
        
        self.trades.append(Trade(
            date=date, type='BUY', price=price,
            shares=shares, value=cost, grid_level=grid['level'] if grid else None
        ))
        
        return True
    
    def execute_sell(self, grid: Dict, price: float, date: pd.Timestamp) -> Optional[Trade]:
        """执行卖出（修复版）"""
        if not grid['active'] or grid['shares'] <= 0:
            return None
        
        shares = grid['shares']
        entry_price = grid['entry_price'] if grid['entry_price'] > 0 else grid['buy_price']
        
        value = shares * price
        self.cash += value
        
        cost = shares * entry_price
        pnl = value - cost
        
        self.position -= shares
        if self.position <= 1e-10:
            self.position = 0
            self.cost_basis = 0
            self._entry_max_price = 0  # 重置ATR止损基准
        
        trade = Trade(
            date=date, type='SELL', price=price,
            shares=shares, value=value, pnl=pnl, grid_level=grid['level']
        )
        self.trades.append(trade)
        
        grid['active'] = False
        grid['shares'] = 0
        grid['entry_price'] = 0
        grid['entry_date'] = None
        
        return trade
    
    def execute_stop_loss(self, price: float, date: pd.Timestamp, stop_type: str = 'STOP_LOSS') -> Trade:
        """执行止损清仓，初始化渐进式重启"""
        if self.position <= 0:
            raise ValueError("持仓为空时不能止损")
        
        value = self.position * price
        self.cash += value
        
        trade = Trade(
            date=date, type=stop_type, price=price,
            shares=self.position, value=value
        )
        self.trades.append(trade)
        
        # 重置状态
        self.position = 0
        self.cost_basis = 0
        self._entry_max_price = 0
        
        # 初始化渐进式重启（分3批，间隔5天）
        self.cooldown_counter = self.config.cooldown_days
        self._restart_phase = 3
        self._restart_base_cash = self.cash * 0.25  # 首批25%
        self._last_restart_idx = self._current_idx
        
        # 重置网格
        if self.grid_manager:
            for grid in self.grid_manager.grids:
                grid['active'] = False
                grid['shares'] = 0
                grid['entry_price'] = 0
        
        return trade
    
    def check_stop_loss(self, total_value: float, price: float, date: pd.Timestamp) -> bool:
        """检查固定比例止损"""
        self.max_value = max(self.max_value, total_value)
        
        if self.max_value <= 0:
            return False
        
        drawdown = (self.max_value - total_value) / self.max_value
        
        if drawdown > self.config.stop_loss and self.position > 0:
            self.execute_stop_loss(price, date, 'STOP_LOSS')
            return True
        
        return False
    
    def check_atr_stop_loss(self, price: float, date: pd.Timestamp) -> bool:
        """ATR-based Chandelier Exit 动态止损"""
        if self.position <= 0 or self._entry_max_price <= 0:
            return False
        
        atr = self._df['ATR'].iloc[self._current_idx]
        stop_price = self._entry_max_price - self.config.atr_stop_multiplier * atr
        
        if price < stop_price:
            self.execute_stop_loss(price, date, 'ATR_STOP')
            return True
        
        return False
    
    def should_update_grids(self, current_price: float) -> bool:
        """判断是否需要更新网格中心"""
        if self.grid_manager is None:
            return True
        
        last_center = self.grid_manager.center_price
        deviation = abs(current_price - last_center) / last_center
        
        return deviation > self.config.position_update_threshold
    
    def should_trend_add(self, i: int, price: float, current_trend: float) -> bool:
        """多因子趋势加仓确认"""
        if current_trend <= 0.7 or self.position <= 0:
            return False
        
        if (i - self._last_add_idx) < 10:
            return False
        
        # 周线趋势过滤
        weekly_trend = self._df['Weekly_Trend'].iloc[i]
        if weekly_trend < 0:
            return False  # 周线向下，禁止加仓
        
        # 价格突破确认
        recent_high = self._df['High'].iloc[max(0, i-20):i].max()
        if price < recent_high * 0.98:
            return False
        
        # 成交量确认
        current_vol = self._df['Volume'].iloc[i]
        avg_vol = self._df['Volume_MA20'].iloc[i]
        if current_vol < avg_vol * 1.2:
            return False
        
        # RSI过滤
        if self._df['RSI'].iloc[i] > 75:
            return False
        
        # 波动率过滤
        atr = self._df['ATR'].iloc[i]
        if atr / price > 0.05:
            return False
        
        return True
    
    def adaptive_rebalance(self, i: int):
        """波动率自适应网格重平衡"""
        if (i - self.last_grid_update_idx) < self.config.rebalance_interval:
            return
        
        if self.grid_manager is None:
            return
        
        recent_vol = self._df['Volatility'].iloc[i]
        current_spacing = self.grid_manager.spacing
        
        # 波动率变化超过50%时重平衡
        if abs(recent_vol - current_spacing) / current_spacing > 0.5:
            new_spacing = max(self.config.min_grid_spacing,
                            min(self.config.max_grid_spacing, recent_vol * 2.5))
            
            # 保留持仓但调整间距
            self.grid_manager.update_center(
                self.grid_manager.center_price, 
                new_spacing, 
                preserve_active=True
            )
            self.last_grid_update_idx = i
    
    def execute_restart_build(self, price: float, date: pd.Timestamp, total_value: float):
        """执行渐进式建仓"""
        if self._restart_phase <= 0:
            return
        
        if (self._current_idx - self._last_restart_idx) < 5:
            return  # 间隔5天
        
        spacing = self.get_grid_spacing(self._df['ATR'].iloc[self._current_idx], price)
        # 减少层级，增加间距
        levels = max(3, self.config.grid_levels // (4 - self._restart_phase))
        wide_spacing = spacing * (1.5 if self._restart_phase == 3 else 1.2)
        
        self.grid_manager = GridManager(price, levels, wide_spacing)
        
        # 买入该批次
        invest = self._restart_base_cash
        shares = invest / price if price > 0 else 0
        
        if shares > 0 and invest <= self.cash * 0.8:  # 保留更多现金
            self.cash -= invest
            self.position += shares
            self.cost_basis = price
            self._entry_max_price = price
            
            self.trades.append(Trade(
                date=date, type='RESTART_BUY', price=price,
                shares=shares, value=invest
            ))
        
        self._restart_phase -= 1
        self._last_restart_idx = self._current_idx
        self.last_grid_update_idx = self._current_idx
    
    def run_backtest(self, df: pd.DataFrame) -> pd.DataFrame:
        """运行回测（优化版）"""
        self.reset()
        
        print("预计算技术指标...")
        self._df = self.prepare_indicators(df)
        
        start_idx = max(self.config.trend_period, self.config.vol_period, 40)
        n_bars = len(self._df)
        
        if n_bars <= start_idx:
            raise ValueError(f"数据长度不足，需要至少{start_idx}条数据")
        
        # 初始化
        init_price = self._df['Close'].iloc[start_idx]
        init_atr = self._df['ATR'].iloc[start_idx]
        spacing = self.get_grid_spacing(init_atr, init_price)
        self.grid_manager = GridManager(init_price, self.config.grid_levels, spacing)
        self.last_grid_update_idx = start_idx
        self._entry_max_price = init_price
        
        print(f"开始回测: {self._df['datetime'].iloc[start_idx]} 至 {self._df['datetime'].iloc[-1]}")
        
        for i in range(start_idx, n_bars):
            self._current_idx = i
            
            date = self._df['datetime'].iloc[i]
            price = self._df['Close'].iloc[i]
            high = self._df['High'].iloc[i]
            low = self._df['Low'].iloc[i]
            
            # 处理冷却期
            if self.cooldown_counter > 0:
                self.cooldown_counter -= 1
                total = self.cash + self.position * price
                self.max_value = max(self.max_value, total)
                self.values.append({
                    'date': date, 'price': price, 'cash': self.cash,
                    'position': self.position, 'value': total,
                    'trend': self._df['Trend'].iloc[i],
                    'weekly_trend': self._df['Weekly_Trend'].iloc[i]
                })
                
                # 渐进式重启建仓
                if self.cooldown_counter <= self.config.cooldown_days - 5:
                    self.execute_restart_build(price, date, total)
                continue
            
            # 波动率自适应重平衡
            self.adaptive_rebalance(i)
            
            # 定期更新网格中心
            current_trend = self._df['Trend'].iloc[i]
            if (i - self.last_grid_update_idx >= 5 or abs(current_trend) > 0.6):
                if self.should_update_grids(price):
                    if abs(current_trend) > 0.3 and self.grid_manager:
                        old_center = self.grid_manager.center_price
                        if current_trend > 0:
                            new_center = max(old_center, price * 0.95)
                        else:
                            new_center = min(old_center, price * 1.05)
                        
                        spacing = self.get_grid_spacing(self._df['ATR'].iloc[i], new_center)
                        closed = self.grid_manager.update_center(new_center, spacing, preserve_active=True)
                        
                        # 处理被迫平仓的网格
                        for grid in closed:
                            if grid['active'] and grid['shares'] > 0:
                                # 以当前价平仓
                                sell_value = grid['shares'] * price
                                self.cash += sell_value
                                self.position -= grid['shares']
                                pnl = sell_value - grid['shares'] * grid['entry_price']
                                self.trades.append(Trade(
                                    date=date, type='GRID_CLOSE', price=price,
                                    shares=grid['shares'], value=sell_value, pnl=pnl
                                ))
                    
                    self.last_grid_update_idx = i
            
            # 计算市值
            position_value = self.position * price
            total_value = self.cash + position_value
            
            # 双重止损检查
            if self.check_stop_loss(total_value, price, date):
                self.values.append({
                    'date': date, 'price': price, 'cash': self.cash,
                    'position': 0, 'value': total_value,
                    'trend': current_trend, 'weekly_trend': self._df['Weekly_Trend'].iloc[i]
                })
                continue
            
            if self.check_atr_stop_loss(price, date):
                self.values.append({
                    'date': date, 'price': price, 'cash': self.cash,
                    'position': 0, 'value': total_value,
                    'trend': current_trend, 'weekly_trend': self._df['Weekly_Trend'].iloc[i]
                })
                continue
            
            # 成交量过滤
            vol_ok = True
            if i >= 20:
                avg_vol = self._df['Volume_MA20'].iloc[i]
                vol_ok = self._df['Volume'].iloc[i] > avg_vol * 0.6
            
            # 网格交易
            if self.grid_manager and vol_ok and current_trend > -0.5:
                buy_signals, sell_signals = self.grid_manager.check_signals(high, low)
                
                # 先处理卖出
                for grid in sell_signals:
                    self.execute_sell(grid, price, date)
                
                # 再处理买入（传入层级信息）
                for grid in buy_signals:
                    self.execute_buy(grid, price, date, total_value)
            
            # 趋势加仓（多因子确认）
            if self.should_trend_add(i, price, current_trend):
                current_pct = self.position * price / total_value
                if current_pct < self.config.max_position * 0.8:
                    inactive = [g for g in self.grid_manager.grids if not g['active']]
                    if inactive:
                        if self.execute_buy(inactive[0], price, date, total_value, is_trend_add=True):
                            self._last_add_idx = i
            
            # 趋势减仓
            if current_trend < -0.8 and self.position > 0:
                # 周线确认
                if self._df['Weekly_Trend'].iloc[i] < 0:
                    reduce_pct = min(0.4, abs(current_trend))
                    reduce_shares = self.position * reduce_pct
                    
                    if reduce_shares > 0:
                        value = reduce_shares * price
                        self.cash += value
                        self.position -= reduce_shares
                        
                        self.trades.append(Trade(
                            date=date, type='TREND_REDUCE', price=price,
                            shares=reduce_shares, value=value
                        ))
            
            # 记录每日数据
            total_value = self.cash + self.position * price
            self.values.append({
                'date': date, 'price': price, 'cash': self.cash,
                'position': self.position, 'value': total_value,
                'trend': current_trend, 'weekly_trend': self._df['Weekly_Trend'].iloc[i]
            })
        
        results = pd.DataFrame(self.values)
        print(f"回测完成: 共{len(results)}个交易日，{len(self.trades)}笔交易")
        return results
    
    def get_performance_metrics(self, results: pd.DataFrame) -> Dict:
        """计算全面绩效指标"""
        if len(results) == 0:
            return {}
        
        initial = self.config.initial_capital
        final = results['value'].iloc[-1]
        total_return = (final - initial) / initial if initial > 0 else 0
        
        days = (results['date'].iloc[-1] - results['date'].iloc[0]).days
        years = days / 365.25 if days > 0 else 0
        annual_return = (1 + total_return) ** (1/years) - 1 if years > 0 else 0
        
        # 最大回撤
        cummax = results['value'].cummax()
        drawdowns = (cummax - results['value']) / cummax
        max_dd = drawdowns.max() if len(cummax) > 0 else 0
        
        # 收益率序列
        daily_ret = results['value'].pct_change().dropna()
        
        # 夏普比率
        if len(daily_ret) > 1 and daily_ret.std() > 0:
            sharpe = ((daily_ret.mean() * 252 - 0.02) / 
                     (daily_ret.std() * np.sqrt(252)))
        else:
            sharpe = 0
        
        # Sortino Ratio（只计算下行波动）
        downside_ret = daily_ret[daily_ret < 0]
        downside_std = downside_ret.std() * np.sqrt(252) if len(downside_ret) > 0 else 0
        sortino = ((daily_ret.mean() * 252 - 0.02) / downside_std) if downside_std > 0 else 0
        
        # 交易统计
        trades_df = pd.DataFrame([{
            'type': t.type, 'pnl': t.pnl, 'value': t.value, 'date': t.date
        } for t in self.trades])
        
        metrics = {
            'initial_capital': initial,
            'final_value': final,
            'total_return_pct': total_return * 100,
            'annual_return_pct': annual_return * 100,
            'max_drawdown_pct': max_dd * 100,
            'sharpe_ratio': sharpe,
            'sortino_ratio': sortino,
            'calmar_ratio': annual_return / max_dd if max_dd > 0 else 0,
            'total_trades': len(self.trades),
        }
        
        if len(trades_df) > 0:
            # 基础统计
            type_counts = trades_df['type'].value_counts().to_dict()
            metrics.update({
                'buy_trades': type_counts.get('BUY', 0),
                'sell_trades': type_counts.get('SELL', 0),
                'stop_loss_trades': type_counts.get('STOP_LOSS', 0) + type_counts.get('ATR_STOP', 0),
                'trend_add_trades': type_counts.get('TREND_ADD', 0),
                'trend_reduce_trades': type_counts.get('TREND_REDUCE', 0),
                'grid_close_trades': type_counts.get('GRID_CLOSE', 0),
                'restart_trades': type_counts.get('RESTART_BUY', 0),
            })
            
            # 盈亏分析
            sell_trades = trades_df[trades_df['type'].isin(['SELL', 'GRID_CLOSE'])]
            if len(sell_trades) > 0:
                pnls = sell_trades['pnl'].dropna()
                wins = pnls[pnls > 0]
                losses = pnls[pnls < 0]
                
                metrics['win_rate_pct'] = len(wins) / len(pnls) * 100 if len(pnls) > 0 else 0
                metrics['avg_pnl'] = pnls.mean()
                metrics['total_pnl'] = pnls.sum()
                metrics['avg_win'] = wins.mean() if len(wins) > 0 else 0
                metrics['avg_loss'] = losses.mean() if len(losses) > 0 else 0
                metrics['profit_factor'] = abs(wins.sum() / losses.sum()) if len(losses) > 0 and losses.sum() != 0 else float('inf')
                metrics['win_loss_ratio'] = abs(metrics['avg_win'] / metrics['avg_loss']) if metrics['avg_loss'] != 0 else 0
                
                # 连续统计
                trade_signs = (pnls > 0).astype(int)
                consecutive_wins = 0
                consecutive_losses = 0
                max_consec_wins = 0
                max_consec_losses = 0
                
                for sign in trade_signs:
                    if sign == 1:
                        consecutive_wins += 1
                        consecutive_losses = 0
                        max_consec_wins = max(max_consec_wins, consecutive_wins)
                    else:
                        consecutive_losses += 1
                        consecutive_wins = 0
                        max_consec_losses = max(max_consec_losses, consecutive_losses)
                
                metrics['max_consecutive_wins'] = max_consec_wins
                metrics['max_consecutive_losses'] = max_consec_losses
            
            # 持仓时间分析
            buy_dates = trades_df[trades_df['type'] == 'BUY'][['date']].copy()
            sell_dates = trades_df[trades_df['type'].isin(['SELL', 'GRID_CLOSE'])][['date']].copy()
            
            if len(buy_dates) > 0 and len(sell_dates) > 0:
                # 简单估算平均持仓天数
                hold_times = []
                for buy_date in buy_dates['date']:
                    future_sells = sell_dates[sell_dates['date'] > buy_date]
                    if len(future_sells) > 0:
                        hold_time = (future_sells['date'].iloc[0] - buy_date).days
                        hold_times.append(hold_time)
                
                metrics['avg_hold_days'] = np.mean(hold_times) if hold_times else 0
        
        return metrics


class StrategyVisualizer:
    """策略可视化（修复时间对齐问题）"""
    
    def __init__(self, config: StrategyConfig):
        self.config = config
        self.colors = {
            'strategy': '#1f77b4', 'buyhold': '#ff7f0e', 'profit': '#2ca02c',
            'loss': '#d62728', 'buy': '#00aa00', 'sell': '#00ff00',
            'stop': '#ff0000', 'atr_stop': '#ff4444', 'add': '#0000ff',
            'reduce': '#ff8800', 'grid_close': '#ff00ff', 'restart': '#00ffff'
        }
    
    def align_data(self, results: pd.DataFrame, df: pd.DataFrame) -> pd.DataFrame:
        """使用merge_asof进行时间对齐"""
        results = results.copy()
        df_aligned = pd.merge_asof(
            results.sort_values('date'),
            df[['datetime', 'Close']].rename(columns={'datetime': 'date', 'Close': 'buyhold_price'}).sort_values('date'),
            on='date',
            direction='backward'
        )
        return df_aligned
    
    def create_comprehensive_report(self, df: pd.DataFrame, 
                                   results: pd.DataFrame,
                                   trades: List[Trade],
                                   output_path: str):
        """生成综合分析报告（修复对齐问题）"""
        trades_df = pd.DataFrame([{
            'date': t.date, 'type': t.type, 'price': t.price,
            'shares': t.shares, 'value': t.value, 'pnl': t.pnl
        } for t in trades])
        
        # 对齐数据
        aligned_results = self.align_data(results, df)
        
        fig = plt.figure(figsize=(20, 28))
        gs = GridSpec(8, 2, height_ratios=[3, 2, 2, 2, 2, 2, 2, 2], 
                     hspace=0.35, wspace=0.25)
        
        # 1. 主图：净值对比
        ax1 = fig.add_subplot(gs[0, :])
        ax1.plot(results['date'], results['value'], 
                label='Trend Grid Strategy', linewidth=2.5, 
                color=self.colors['strategy'], alpha=0.9)
        
        # 修复后的买入持有基准计算
        if 'buyhold_price' in aligned_results.columns and aligned_results['buyhold_price'].notna().any():
            first_valid = aligned_results['buyhold_price'].first_valid_index()
            if first_valid is not None:
                initial_price = aligned_results.loc[first_valid, 'buyhold_price']
                buyhold_values = aligned_results['buyhold_price'] / initial_price * self.config.initial_capital
                
                ax1.plot(aligned_results['date'], buyhold_values, 
                        label='Buy & Hold', linewidth=2, 
                        color=self.colors['buyhold'], alpha=0.7, linestyle='--')
                
                buyhold_final = buyhold_values.iloc[-1]
        else:
            buyhold_final = self.config.initial_capital
        
        ax1.axhline(y=self.config.initial_capital, color='gray', 
                   linestyle=':', alpha=0.5, linewidth=1)
        ax1.fill_between(results['date'], self.config.initial_capital, results['value'],
                        where=(results['value'] >= self.config.initial_capital), 
                        alpha=0.2, color=self.colors['profit'])
        ax1.fill_between(results['date'], self.config.initial_capital, results['value'],
                        where=(results['value'] < self.config.initial_capital), 
                        alpha=0.2, color=self.colors['loss'])
        
        final_value = results['value'].iloc[-1]
        strategy_return = (final_value / self.config.initial_capital - 1) * 100
        buyhold_return = (buyhold_final / self.config.initial_capital - 1) * 100
        
        ax1.set_title(f'Trend-Enhanced Smart Grid Strategy v4.0\n' + 
                     f'Strategy: {strategy_return:.1f}% vs Buy&Hold: {buyhold_return:.1f}%',
                     fontsize=16, fontweight='bold', pad=20)
        ax1.set_ylabel('Portfolio Value (CNY)', fontsize=12)
        ax1.legend(loc='upper left', fontsize=11)
        ax1.grid(True, alpha=0.3, linestyle='--')
        
        ax1.annotate(f'Strategy\n{final_value:,.0f}\n({strategy_return:.1f}%)', 
                    xy=(results['date'].iloc[-1], final_value), 
                    xytext=(-120, 20), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor=self.colors['strategy'], alpha=0.8),
                    fontsize=10, color='white', fontweight='bold')
        
        if 'buyhold_price' in aligned_results.columns:
            ax1.annotate(f'Buy&Hold\n{buyhold_final:,.0f}\n({buyhold_return:.1f}%)', 
                        xy=(results['date'].iloc[-1], buyhold_final), 
                        xytext=(-120, -60), textcoords='offset points',
                        bbox=dict(boxstyle='round,pad=0.5', facecolor=self.colors['buyhold'], alpha=0.8),
                        fontsize=10, color='white', fontweight='bold')
        
        # 2. 价格与交易点
        ax2 = fig.add_subplot(gs[1, :])
        ax2.plot(df['datetime'], df['Close'], label='Stock Price', 
                color='black', linewidth=1.5, alpha=0.8)
        
        if len(trades_df) > 0:
            for trade_type, marker, color_key in [
                ('BUY', '^', 'buy'), ('SELL', 'v', 'sell'),
                ('STOP_LOSS', 'x', 'stop'), ('ATR_STOP', 'X', 'atr_stop'),
                ('TREND_ADD', '*', 'add'), ('TREND_REDUCE', 'D', 'reduce'),
                ('GRID_CLOSE', 'p', 'grid_close'), ('RESTART_BUY', 'h', 'restart')
            ]:
                subset = trades_df[trades_df['type'] == trade_type]
                if len(subset) > 0:
                    ax2.scatter(subset['date'], subset['price'], 
                               marker=marker, s=100, color=self.colors[color_key],
                               alpha=0.8, label=f'{trade_type} ({len(subset)})', zorder=5, edgecolors='black', linewidth=0.5)
        
        ax2.set_title('Stock Price & Trading Signals', fontsize=14, fontweight='bold')
        ax2.set_ylabel('Price (CNY)', fontsize=11)
        ax2.legend(loc='upper left', ncol=4, fontsize=9)
        ax2.grid(True, alpha=0.3)
        
        # 3. 持仓比例
        ax3 = fig.add_subplot(gs[2, 0])
        position_pct = results['position'] * results['price'] / results['value'] * 100
        ax3.fill_between(results['date'], 0, position_pct, alpha=0.6, color='steelblue')
        ax3.plot(results['date'], position_pct, color='navy', linewidth=1)
        ax3.axhline(y=self.config.max_position*100, color='darkred', 
                   linestyle='--', alpha=0.3, label=f'Max {self.config.max_position*100:.0f}%')
        ax3.set_ylabel('Position Ratio (%)', fontsize=11)
        ax3.set_title('Position Allocation', fontsize=13, fontweight='bold')
        ax3.legend(loc='upper right')
        ax3.grid(True, alpha=0.3)
        ax3.set_ylim(0, 100)
        
        # 4. 现金余额与储备线
        ax4 = fig.add_subplot(gs[2, 1])
        cash_reserve_line = results['value'] * self.config.min_cash_reserve
        ax4.fill_between(results['date'], 0, results['cash'], 
                        alpha=0.5, color='green', label='Cash')
        ax4.plot(results['date'], results['cash'], 
                color='darkgreen', linewidth=1.5)
        ax4.plot(results['date'], cash_reserve_line, 
                color='red', linestyle='--', alpha=0.5, label='Reserve Line')
        ax4.set_ylabel('Cash (CNY)', fontsize=11)
        ax4.set_title('Cash Reserve', fontsize=13, fontweight='bold')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        # 5. 趋势强度（日线+周线）
        ax5 = fig.add_subplot(gs[3, 0])
        ax5.plot(results['date'], results['trend'], 
                color='purple', linewidth=1.5, label='Daily Trend')
        if 'weekly_trend' in results.columns:
            ax5.plot(results['date'], results['weekly_trend'], 
                    color='blue', linewidth=1, alpha=0.7, linestyle='--', label='Weekly Trend')
        ax5.axhline(y=0.5, color='green', linestyle='--', alpha=0.5)
        ax5.axhline(y=-0.5, color='red', linestyle='--', alpha=0.5)
        ax5.fill_between(results['date'], -1, 1, 
                        where=(results['trend'] > 0.5), alpha=0.2, color='green')
        ax5.fill_between(results['date'], -1, 1, 
                        where=(results['trend'] < -0.5), alpha=0.2, color='red')
        ax5.set_ylabel('Trend Strength', fontsize=11)
        ax5.set_title('Trend Indicator (Daily + Weekly)', fontsize=13, fontweight='bold')
        ax5.legend()
        ax5.grid(True, alpha=0.3)
        ax5.set_ylim(-1.2, 1.2)
        
        # 6. RSI
        ax6 = fig.add_subplot(gs[3, 1])
        ax6.plot(df['datetime'], df['RSI'], color='purple', linewidth=1)
        ax6.axhline(y=70, color='red', linestyle='--', alpha=0.5)
        ax6.axhline(y=30, color='green', linestyle='--', alpha=0.5)
        ax6.fill_between(df['datetime'], 30, 70, alpha=0.1, color='gray')
        ax6.set_ylabel('RSI', fontsize=11)
        ax6.set_title('RSI Indicator (14-day)', fontsize=13, fontweight='bold')
        ax6.grid(True, alpha=0.3)
        ax6.set_ylim(0, 100)
        
        # 7. 回撤
        ax7 = fig.add_subplot(gs[4, 0])
        cummax = results['value'].cummax()
        drawdown = (cummax - results['value']) / cummax * 100
        ax7.fill_between(results['date'], 0, drawdown, 
                        alpha=0.5, color='red')
        ax7.plot(results['date'], drawdown, 
                color='darkred', linewidth=1.5)
        
        max_dd = drawdown.max()
        max_dd_idx = drawdown.idxmax()
        ax7.scatter(results['date'].iloc[max_dd_idx], max_dd, 
                   s=100, color='darkred', zorder=5)
        ax7.annotate(f'Max DD: {max_dd:.1f}%', 
                    xy=(results['date'].iloc[max_dd_idx], max_dd),
                    xytext=(10, 10), textcoords='offset points',
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8),
                    fontsize=10, fontweight='bold')
        ax7.set_ylabel('Drawdown (%)', fontsize=11)
        ax7.set_title('Drawdown Analysis', fontsize=13, fontweight='bold')
        ax7.grid(True, alpha=0.3)
        
        # 8. 成交量
        ax8 = fig.add_subplot(gs[4, 1])
        price_change = df['Close'].diff().fillna(0)
        colors_vol = ['red' if c >= 0 else 'green' for c in price_change]
        ax8.bar(df['datetime'], df['Volume']/1e6, 
               color=colors_vol, alpha=0.6, width=1)
        ax8.set_ylabel('Volume (M)', fontsize=11)
        ax8.set_title('Trading Volume', fontsize=13, fontweight='bold')
        ax8.grid(True, alpha=0.3)
        
        # 9. 盈亏分布
        ax9 = fig.add_subplot(gs[5, 0])
        sell_trades = trades_df[trades_df['type'].isin(['SELL', 'GRID_CLOSE'])]
        if len(sell_trades) > 0 and 'pnl' in sell_trades.columns:
            pnls = sell_trades['pnl'].dropna()
            if len(pnls) > 0:
                colors_pnl = ['green' if p > 0 else 'red' for p in pnls]
                ax9.bar(range(len(pnls)), pnls, color=colors_pnl, alpha=0.7)
                ax9.axhline(y=0, color='black', linestyle='-', linewidth=1)
                
                win_rate = (pnls > 0).sum() / len(pnls) * 100
                ax9.set_xlabel('Trade Number', fontsize=11)
                ax9.set_ylabel('PnL (CNY)', fontsize=11)
                ax9.set_title(f'PnL Distribution (Win Rate: {win_rate:.1f}%)', 
                            fontsize=13, fontweight='bold')
                ax9.grid(True, alpha=0.3, axis='y')
        
        # 10. 交易统计
        ax10 = fig.add_subplot(gs[5, 1])
        trade_types = ['BUY', 'SELL', 'STOP_LOSS', 'ATR_STOP', 'TREND_ADD', 'TREND_REDUCE', 'GRID_CLOSE', 'RESTART_BUY']
        trade_counts = [len(trades_df[trades_df['type']==t]) for t in trade_types]
        colors_type = ['green', 'lime', 'red', 'darkred', 'blue', 'orange', 'magenta', 'cyan']
        bars = ax10.bar([t.replace('_', '\n') for t in trade_types], 
                       trade_counts, color=colors_type, alpha=0.7)
        ax10.set_ylabel('Count', fontsize=11)
        ax10.set_title('Trade Statistics', fontsize=13, fontweight='bold')
        
        for bar, count in zip(bars, trade_counts):
            if count > 0:
                ax10.text(bar.get_x() + bar.get_width()/2, 
                         bar.get_height() + max(trade_counts)*0.01, 
                         str(count), ha='center', va='bottom', 
                         fontsize=9, fontweight='bold')
        ax10.grid(True, alpha=0.3, axis='y')
        
        # 11. 滚动夏普比率
        ax11 = fig.add_subplot(gs[6, 0])
        rolling_returns = results['value'].pct_change()
        rolling_sharpe = (rolling_returns.rolling(60).mean() * 252 - 0.02) / (rolling_returns.rolling(60).std() * np.sqrt(252))
        ax11.plot(results['date'], rolling_sharpe, color='blue', linewidth=1)
        ax11.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        ax11.axhline(y=1, color='green', linestyle='--', alpha=0.3)
        ax11.set_ylabel('Sharpe Ratio', fontsize=11)
        ax11.set_title('Rolling Sharpe Ratio (60-day)', fontsize=13, fontweight='bold')
        ax11.grid(True, alpha=0.3)
        
        # 12. 月度收益热力图
        ax12 = fig.add_subplot(gs[6, 1])
        results['month'] = results['date'].dt.to_period('M')
        monthly_returns = results.groupby('month')['value'].apply(lambda x: (x.iloc[-1] / x.iloc[0] - 1) * 100)
        
        if len(monthly_returns) > 0:
            # 转换为二维热力图数据
            monthly_df = monthly_returns.reset_index()
            monthly_df['year'] = monthly_df['month'].dt.year
            monthly_df['mon'] = monthly_df['month'].dt.month
            
            pivot_table = monthly_df.pivot(index='year', columns='mon', values='value')
            
            im = ax12.imshow(pivot_table.values, cmap='RdYlGn', aspect='auto', vmin=-10, vmax=10)
            ax12.set_xticks(range(12))
            ax12.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
            ax12.set_yticks(range(len(pivot_table)))
            ax12.set_yticklabels(pivot_table.index)
            ax12.set_title('Monthly Returns Heatmap (%)', fontsize=13, fontweight='bold')
            plt.colorbar(im, ax=ax12)
        
        # 13. ATR与网格间距
        ax13 = fig.add_subplot(gs[7, 0])
        ax13.plot(df['datetime'], df['ATR'] / df['Close'] * 100, 
                color='orange', linewidth=1, label='ATR %')
        ax13.axhline(y=self.config.base_grid_pct*100, color='blue', 
                    linestyle='--', alpha=0.5, label='Base Grid %')
        ax13.set_ylabel('ATR / Price (%)', fontsize=11)
        ax13.set_title('Volatility vs Grid Spacing', fontsize=13, fontweight='bold')
        ax13.legend()
        ax13.grid(True, alpha=0.3)
        
        # 14. 资金曲线与回撤叠加
        ax14 = fig.add_subplot(gs[7, 1])
        ax14_twin = ax14.twinx()
        
        ax14.plot(results['date'], results['value'], color='blue', linewidth=1.5, label='Portfolio Value')
        ax14_twin.fill_between(results['date'], 0, drawdown, alpha=0.3, color='red', label='Drawdown')
        
        ax14.set_ylabel('Value (CNY)', color='blue', fontsize=11)
        ax14_twin.set_ylabel('Drawdown (%)', color='red', fontsize=11)
        ax14.set_title('Equity Curve with Drawdown', fontsize=13, fontweight='bold')
        ax14.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        output_file = Path(output_path) / 'comprehensive_analysis_v4.png'
        plt.savefig(output_file, dpi=200, bbox_inches='tight', 
                   facecolor='white', edgecolor='none')
        plt.close()
        
        print(f"✅ 综合分析图表已保存: {output_file}")
        return output_file


In [10]:
def print_performance_report(metrics: Dict):
    """打印格式化绩效报告"""
    print("\n" + "="*80)
    print(" " * 25 + "策略绩效报告 v4.0")
    print("="*80)
    
    print(f"{'初始资金:':<25} {metrics.get('initial_capital', 0):>15,.0f} CNY")
    print(f"{'最终价值:':<25} {metrics.get('final_value', 0):>15,.0f} CNY")
    print(f"{'总收益率:':<25} {metrics.get('total_return_pct', 0):>15.2f} %")
    print(f"{'年化收益:':<25} {metrics.get('annual_return_pct', 0):>15.2f} %")
    print(f"{'最大回撤:':<25} {metrics.get('max_drawdown_pct', 0):>15.2f} %")
    print("-"*80)
    print(f"{'夏普比率:':<25} {metrics.get('sharpe_ratio', 0):>15.2f}")
    print(f"{'索提诺比率:':<25} {metrics.get('sortino_ratio', 0):>15.2f}")
    print(f"{'卡玛比率:':<25} {metrics.get('calmar_ratio', 0):>15.2f}")
    print(f"{'盈亏比:':<25} {metrics.get('win_loss_ratio', 0):>15.2f}")
    print(f"{'盈利因子:':<25} {metrics.get('profit_factor', 0):>15.2f}")
    print("-"*80)
    print(f"{'总交易次数:':<25} {metrics.get('total_trades', 0):>15}")
    print(f"{'买入次数:':<25} {metrics.get('buy_trades', 0):>15}")
    print(f"{'卖出次数:':<25} {metrics.get('sell_trades', 0):>15}")
    print(f"{'止损次数 (固定):':<25} {metrics.get('stop_loss_trades', 0):>15}")
    print(f"{'ATR止损次数:':<25} {metrics.get('atr_stop_trades', 0):>15}")
    print(f"{'网格强制平仓:':<25} {metrics.get('grid_close_trades', 0):>15}")
    print(f"{'趋势加仓:':<25} {metrics.get('trend_add_trades', 0):>15}")
    print(f"{'趋势减仓:':<25} {metrics.get('trend_reduce_trades', 0):>15}")
    print(f"{'重启建仓:':<25} {metrics.get('restart_trades', 0):>15}")
    print("-"*80)
    print(f"{'胜率:':<25} {metrics.get('win_rate_pct', 0):>15.1f} %")
    print(f"{'平均盈亏:':<25} {metrics.get('avg_pnl', 0):>15,.0f} CNY")
    print(f"{'平均盈利:':<25} {metrics.get('avg_win', 0):>15,.0f} CNY")
    print(f"{'平均亏损:':<25} {metrics.get('avg_loss', 0):>15,.0f} CNY")
    print(f"{'总盈亏:':<25} {metrics.get('total_pnl', 0):>15,.0f} CNY")
    print(f"{'最大连续盈利:':<25} {metrics.get('max_consecutive_wins', 0):>15}")
    print(f"{'最大连续亏损:':<25} {metrics.get('max_consecutive_losses', 0):>15}")
    print(f"{'平均持仓天数:':<25} {metrics.get('avg_hold_days', 0):>15.1f}")
    print("="*80)


In [11]:
def main():
    """主函数"""
    DATA_PATH = 'C:/Users/1/Desktop/python量化/603993历史数据(2020-2025).csv'
    OUTPUT_PATH = 'C:/Users/1/Desktop'
    
    print("="*80)
    print("趋势增强型智能网格策略 v4.0 - 全面优化版")
    print("="*80)
    
    print("\n1. 加载数据...")
    try:
        df = pd.read_csv(DATA_PATH)
        print(f"   原始数据: {len(df)} 行")
    except Exception as e:
        print(f"   ❌ 数据加载失败: {e}")
        return
    
    print("\n2. 初始化策略...")
    config = StrategyConfig(
        initial_capital=100000,
        base_grid_pct=0.035,  # 调整为3.5%
        grid_levels=5,        # 调整为5层
        max_position=0.9,
        stop_loss=0.12,       # 调整为12%
        cooldown_days=12,     # 调整为12天
        position_update_threshold=0.08,  # 调整为8%
        min_cash_reserve=0.1  # 新增：保留10%现金
    )
    
    strategy = TrendEnhancedGridStrategy(config)
    
    print("\n3. 运行回测...")
    try:
        results = strategy.run_backtest(df)
    except Exception as e:
        print(f"   ❌ 回测失败: {e}")
        import traceback
        traceback.print_exc()
        return
    
    print("\n4. 计算绩效指标...")
    metrics = strategy.get_performance_metrics(results)
    print_performance_report(metrics)
    
    print("\n5. 生成可视化图表...")
    try:
        visualizer = StrategyVisualizer(config)
        visualizer.create_comprehensive_report(
            strategy._df, results, strategy.trades, OUTPUT_PATH
        )
    except Exception as e:
        print(f"   ⚠️ 图表生成失败: {e}")
        import traceback
        traceback.print_exc()
    
    print("\n" + "="*80)
    print("✅ 回测完成！")
    print(f"输出目录: {OUTPUT_PATH}")
    print("="*80)


In [12]:
if __name__ == "__main__":
    main()

趋势增强型智能网格策略 v4.0 - 全面优化版

1. 加载数据...
   原始数据: 1473 行

2. 初始化策略...

3. 运行回测...
预计算技术指标...
开始回测: 2020-03-24 00:00:00 至 2026-02-13 00:00:00
回测完成: 共1433个交易日，296笔交易

4. 计算绩效指标...

                         策略绩效报告 v4.0
初始资金:                             100,000 CNY
最终价值:                             128,928 CNY
总收益率:                               28.93 %
年化收益:                                4.41 %
最大回撤:                               38.16 %
--------------------------------------------------------------------------------
夏普比率:                                0.24
索提诺比率:                               0.26
卡玛比率:                                0.12
盈亏比:                                 1.80
盈利因子:                                5.40
--------------------------------------------------------------------------------
总交易次数:                                296
买入次数:                                   6
卖出次数:                                   4
止损次数 (固定):                             96
ATR止损次数:                